# 第7讲：视觉里程计1


## 7.1 特征点法

视觉SLAM分为视觉前端和优化后端。前端也称为视觉里程计（VO），根据相邻图像信息，粗略估计相机运动，为后端提供初始值。

VO的实现按是否需要提取特征，分为特征点法前端及不提特征的直接法前端。基于特征点法的前端运行稳定，对光照、动态物体不敏感，是比较成熟的解决方。

### 7.1.1 特征点

VO主要问题是根据图像来估计相机运动：

1. 从图像中选取代表性点，要求当相机视角发生少许变化时，这些点保持不变；

2. 根据这些点求解相机位姿估计问题及这些点的定位问题。

在经典SLAM模型中，这些代表性点称为**路标**。视觉SLAM中，路标指**图像特征（features）**。*特征是图像信息的一种数字表达形式*。视觉里程计要求*在相机运动前后，特征点保持稳定*，即当场景、相机视角发生少许改变时，仍能从图像中判断哪些像素属于同一位置。数字图像在计算机中以灰度值矩阵方式存储，因此图像像素是一种最简单的特征，但其易受光照、形变、物体材质影响，在不同图像之间变化非常大，不够稳定。

**特征点**：图像中*特别的地方*，角点、边缘和区块均可视为图像的代表性地方。其中，角点最容易识别；其次为边缘（图像局部沿着边缘相似）；区块最难识别。图像中的角点、边缘相比像素区块更“特别”，在不同图像之间辨识度更高。

<img src="./img/fig_7_1.png" width="500" />

在大多数应用中，单纯角点无法满足需求。例如，从远处看是角点的地方，当相机移近后，可能显示为非角点；或者当相机旋转后，角点外观会发生变化，也无法辨认出同一角点。

局部图像特征性质：

1. 可重复性（repeatability）：相同“区域”可在不同图像中被辩识；

2. 可区别性（distinctiveness）：不同“区域”的表达不同；

3. 高效率（efficiency）：同一图像特征点数量远小于像素数量；

4. 本地性（locality）：特征仅与图像局部区域相关。

特征点由**关键点（key-point）**和**描述子（descriptor）**两部分组成。关键点：该特征点在图像中的位置（方向、大小）等信息。描述子：通常为向量，按照某种人工设计方式，描述该关键点周围像素的信息。描述子是按照“*外观相似的特征应该有相似的描述子*”的原则设计的。两个特征点的描述子在向量空间上的距离相近即表示它们是同样的特征点。

常用局部图像特征：`SIFT`、`SURF`、`ORB`。SIFT（尺度不变特征变换，scale-invariant feature transform）最为经典，其充分考虑图像变换过程中出现的光照、尺度、旋转等变化，但计算量极大，因此在SLAM中很少使用。`FAST`关键点计算量极小（只有关键点，没有描述子）。ORB（oriented FAST and rotated BRIEF）特征是一种实时图像特征，其针对FAST关键点不具备方向性的问题进行改进，并采用二进制描述子BRIEF，使图像特征提取速度极快。


### 7.1.2 ORB特征

ORB特征由关键点（oriented FAST）和描述子（BRIEF，binary robust independent elementary features）两部分组成，提取ORB特征分为两个步骤：

1. FAST角点提取：找出图像中的“角点”。相比原版FAST，ORB计算特征点的主方向，为BRIEF描述子增加旋转不变特性。

2. BRIEF描述子：对提取的特征点的周围图像区域进行描述。

<img src="./img/fig_7_2.png" width="500" />

* **FAST关键点**

FAST是一种角点，主要检测局部像素灰度变化明显的地方，速度极快。其基本思想为：如果一个像素与它邻域像素差别较大（过亮或过暗），则它更可能是角点。相比于其他角点检测算法，FAST只需比较像素亮度。其检测过程如下：

1. 在图像中选取像素$p$，假设其亮度为$I_{p}$。

2. 设置一个阈值$T$（例如$I_{p}$的20%）。

3. 以像素$p$为中心，取半径为3的圆上的16个像素点。

4. 假如选取的圆上，有连续$N$个点的亮度大于$I_{p} + T$或小于$I_{p} - T$，则像素$p$为特征点（$N$通常取12，即FAST-12。其它常用N的取值为9和11，分别称为FAST-9、FAST-11）。

5. 循环以上四步，对每个像素执行相同的操作。

<img src="./img/fig_7_3.png" width="500" />

在FAST-12算法中，为了进一步提高效率，可以添加预测试操作，以快速排除绝大多数非角点像素。具体操作为：对每个像素直接检测邻域圆上的第1、5、9、13个像素的亮度，仅当这四个像素中有三个同时大于$I_{p} + T$或小于$I_{p} - T$时，该像素才可能是一个角点，否则直接排除。此外，原始FAST角点经常出现“扎堆”现象，因此在第一轮检测后，还需要非极大值抑制（non-maximal suppression），仅保留区域内响应极大值的角点，避免角点集中的问题。

FAST特征点存在的问题：

1. FAST特征点数量巨大且不确定，通常需要从图像中提取数量固定的特征。因此，ORB对原始FAST算法进行了改进。指定需要提取角点数量$N$，对原始FAST角点分别计算Harris响应值，然后选取前$N$个具有最大响应值的角点作为最终角点集合。

2. FAST角点不具备方向信息。

3. 领域取半径固定为3圆，存在尺度问题：远处看是角点的地方，接近后可能不是角点。

针对FAST角点缺少方向性和尺度问题，ORB添加尺度和旋转描述。尺度不变性通过构建图像金字塔并在金字塔各层上检测角点实现；特征旋转由灰度质心法（intensity centroid）实。

**质心**：以图像块灰度值作为权重中心，具体操作步骤为：

1. 图像块$B$中，定义图像块的矩为：

$$m_{pq} = \sum_{x, y \in B} x^{p} y^{q} I(x, y), \ p, q = \{ 0, 1 \}$$

2. 通过矩计算图像块质心：

$$C = \left( \frac{m_{10}}{m_{00}}, \frac{m_{01}}{m_{00}} \right)$$

3. 连接图像块几何中心$O$与质心$C$，得到方向向量$\vec{OC}$，则特征点的方向定义为：

$$\theta = \arctan \left( \frac{m_{01}}{m_{10}} \right)$$

通过上述方法，使FAST角点具备尺度与旋转的描述，提升其在不同图像之间表述的鲁棒性。ORB将改进后的FAST称为oriented FAST。

* **BRIEF描述子**

BRIEF是一种二进制描述子，其描述向量由多个0和1组成，这里的0和1编码了关键点附近两个像素（比如说 p 和 q）的大小关系：如果 p 比 q 大，则取 1，反之就
取 0。如果我们取了 128 个这样的 p, q，最后就得到 128 维由 0，1 组成的向量。那么，p
和 q 如何选取呢？在作者原始的论文中给出了若干种挑选方法，大体上都是按照某种概率
分布，随机地挑选 p 和 q 的位置，读者可以阅读 BRIEF 论文或 OpenCV 源码以查看它
的具体实现 [34]。BRIEF 使用了随机选点的比较，速度非常快，而且由于使用了二进制表
达，存储起来也十分方便，适用于实时的图像匹配。原始的 BRIEF 描述子不具有旋转不
变性的，因此在图像发生旋转时容易丢失。而 ORB 在 FAST 特征点提取阶段计算了关键
点的方向，所以可以利用方向信息，计算了旋转之后的“Steer BRIEF”特征，使 ORB 的
描述子具有较好的旋转不变性。
由于考虑到了旋转和缩放，使得 ORB 在平移、旋转、缩放的变换下仍有良好的表现。
同时，FAST 和 BRIEF 的组合也非常的高效，使得 ORB 特征在实时 SLAM 中非常受欢
迎。我们在图 7-2 中展示了一张图像提取 ORB 之后的结果，下面来介绍如何在不同的图
像之间进行特征匹配。


### 7.1.3 特征匹配

<img src="./img/fig_7_4.png" width="500" />

## 7.2 实践：特征提取和匹配


In [16]:
# feature_extraction.py

import cv2

def orb_op(image, orb):
    image = image.copy()
    # 第一步：检测Oriented FAST角点位置
    keypoints = orb.detect(image=image)
    # 第二步：根据角点位置计算BRIEF描述子
    descriptors = orb.compute(
        image=image, keypoints=keypoints
    )
    return keypoints, descriptors

if __name__ =="__main__":
    
    # 读取图像
    image_1 = cv2.imread("./chapter_07/1.png")
    image_2 = cv2.imread("./chapter_07/2.png")
    
    # 初始化
    orb = cv2.ORB_create(
        nfeatures=500,
        scaleFactor=1.2,
        nlevels=8,
        edgeThreshold=31,
        firstLevel=0,
        WTA_K=2,
        scoreType=cv2.ORB_HARRIS_SCORE,
        patchSize=31,
        fastThreshold=20
    )
    
    keypoints_1, descriptors_1 = orb_op(image=image_1, orb=orb)
    keypoints_2, descriptors_2 = orb_op(image=image_2, orb=orb)
    
    output_1 = image_1.copy()
    cv2.drawKeypoints(
        image=image_1,
        keypoints=keypoints_1,
        outImage=output_1,
        flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT
        # flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS
    )
    
    cv2.imshow("ORB特征点", output_1)
    cv2.waitKey(0)
    
    cv2.destroyAllWindows()

    # 第三步：对两幅图像中的BRIEF描述子进行匹配，使用Hamming距离
    matcher = cv2.BFMatcher_create(
        normType=cv2.NORM_HAMMING
    )
    matches = matcher.match(
        descriptors_1, descriptors_2
    )


TypeError: Expected Ptr<cv::UMat> for argument '%s'

图 7-6 显示了例程的运行结果。我们看到未筛选的匹配中带有大量的误匹配。经过一
次筛选之后，匹配数量减少了许多，但大多数匹配都是正确的。这里，我们筛选的依据是汉
明距离小于最小距离的两倍，这是一种工程上的经验方法，不一定有理论依据。不过，尽
管在示例图像中能够筛选出正确的匹配，但我们仍然不能保证在所有其他图像中得到的匹
配全是正确的。因此，在后面的运动估计中，还需要使用去除误匹配的算法。
接下来，我们希望根据匹配的点对，估计相机的运动。这里由于相机的原理不同，情
况发生了变化：